# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,304.976212,-0.907610,1234,304.068602,305.883822,608.137204,611.767644
6,81.819243,0.425770,1234,82.245013,81.393473,164.490026,162.786946
7,691.419938,-0.998698,1234,690.421240,692.418636,1380.842479,1384.837273
8,377.263434,0.601367,1234,377.864802,376.662067,755.729603,753.324133
9,697.149163,-0.144065,1234,697.005098,697.293227,1394.010196,1394.586455
10,437.643516,-0.468869,1234,437.174647,438.112385,874.349295,876.224770
11,436.226417,0.205031,1234,436.431448,436.021385,872.862895,872.042771
12,115.749289,0.968024,1234,116.717313,114.781264,233.434626,229.562529
13,894.028431,-0.044819,1234,893.983611,894.073250,1787.967223,1788.146501
14,310.504710,-0.694461,1234,309.810249,311.199170,619.620498,622.398340


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-304.068602,305.883822
6,-82.245013,81.393473
7,-690.421240,692.418636
8,-377.864802,376.662067
9,-697.005098,697.293227
10,-437.174647,438.112385
11,-436.431448,436.021385
12,-116.717313,114.781264
13,-893.983611,894.073250
14,-309.810249,311.199170


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-304.068602,305.883822
6,-82.245013,81.393473
7,-690.421240,692.418636
8,-377.864802,376.662067
9,-697.005098,697.293227
10,-437.174647,438.112385
11,-436.431448,436.021385
12,-116.717313,114.781264
13,-893.983611,894.073250
14,-309.810249,311.199170


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-304.06860182,  305.88382185],
       [ -82.24501294,   81.39347313],
       [-690.42123975,  692.41863629],
       [-377.86480173,  376.66206674],
       [-697.00509798,  697.29322748],
       [-437.17464738,  438.11238504],
       [-436.43144774,  436.02138528],
       [-116.71731306,  114.78126443],
       [-893.98361128,  894.07325028],
       [-309.81024894,  311.1991702 ]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,606.137204,-611.767644
6,162.490026,-162.786946
7,1378.842479,-1384.837273
8,753.729603,-753.324133
9,1392.010196,-1394.586455
10,872.349295,-876.224770
11,870.862895,-872.042771
12,231.434626,-229.562529
13,1785.967223,-1788.146501
14,617.620498,-622.398340


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
